<a href="https://colab.research.google.com/github/Theoph-ay/pytorch_tutorials/blob/main/05_Going_Modular_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path

Path("going_modular/").mkdir(parents=True, exist_ok=True)

In [ ]:
%%writefile going_modular/get_data.py
"""
Takes a url and downloads the data into the specified folder
"""

import os
import requests
from pathlib import Path
import zipfile


def get_data(
    data_path: Path,
    image_path: Path,
    url: str
):

  """
  Takes a url and downloads the data into the specified folder
  Checks if the data exists before downloading

  Args:
      data_path (Path): Path to the data folder
      image_path (Path): Path to the image folder
      url (str): URL to the data

  """
  # Check if the image folder exists
  if (image_path/ "train").is_dir():
      print(f"{image_path} / train directory exists. Skipping download.")
  else:
      print(f"Did not find {image_path} /train directory, creating one...")
      image_path.mkdir(parents=True, exist_ok=True)

      # Download pizza, steak, sushi data
      zip_save_path = data_path / "pizza_steak_sushi.zip"
      with open(zip_save_path, "wb") as f:
          print("Downloading pizza, steak, sushi data...")
          request = requests.get(url)
          f.write(request.content)

      # Unzip pizza, steak, sushi data
      with zipfile.ZipFile(zip_save_path, "r") as zip_ref:
          print("Unzipping pizza, steak, sushi data...")
          zip_ref.extractall(image_path)

      # Remove zip file
      os.remove(zip_save_path)
      print("Download and extraction complete.")

Writing going_modular/get_data.py


In [ ]:
from pathlib import Path
from going_modular import get_data

data_path = Path("data")
image_path = data_path / "pizza_steak_sushi"
get_data.get_data(
    data_path=data_path,
    image_path=image_path,
    url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
)

Did not find data/pizza_steak_sushi /train directory, creating one...
Unzipping pizza, steak, sushi data...
Download and extraction complete.


In [ ]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """
  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False, # don't need to shuffle test data
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names
#If we'd like to make DataLoader's we can now use the function within data_setup.py like so:

# Import data_setup.py
#from going_modular import data_setup

# Create train/test dataloader and get class names as a list
#train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(...)

Writing going_modular/data_setup.py


In [ ]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
      )

  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x

Writing going_modular/model_builder.py


In [ ]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item()

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(dataloader):
          # Send data to target device
          X, y = X.to(device), y.to(device)

          # 1. Forward pass
          test_pred_logits = model(X)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          test_pred_labels = test_pred_logits.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

Writing going_modular/engine.py


In [ ]:
%%writefile going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing going_modular/utils.py


In [ ]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

import argparse
# Setup hyperparameters with argparse
parser = argparse.ArgumentParser(description = "Getting hyperparameters")

parser.add_argument("-ne", "--num_epochs", type=int, default=5, help = "Number of epochs to train for")
parser.add_argument("-bs", "--batch_size", type=int, default=32, help = "DataLoader Batch size")
parser.add_argument("-lr", "--learning_rate", type=float, default = 0.001, help = "Learning rate of the optimizer")
parser.add_argument("-hu", "--hidden_units", type=int, default = 10, help = "Number of hidden units in the convolutional neural network")
parser.add_argument("--trd", default ="data/pizza_steak_sushi/train", help = "Train data directory")
parser.add_argument("--td", default = "data/pizza_steak_sushi/test", help = "Test data directory")

args = parser.parse_args()


# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=args.trd,
    test_dir=args.td,
    transform=data_transform,
    batch_size=args.batch_size
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=args.hidden_units,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=args.learning_rate)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=args.num_epochs,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Writing going_modular/train.py


In [ ]:
!python going_modular/train.py --learning_rate 0.003 --batch_size 64 --num_epochs 20

  0% 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch: 1 | train_loss: 1.1021 | train_acc: 0.3414 | test_loss: 1.0836 | test_acc: 0.3253
  5% 1/20 [00:02<00:38,  2.02s/it]Epoch: 2 | train_loss: 1.0930 | train_acc: 0.4068 | test_loss: 1.0182 | test_acc: 0.6733
 10% 2/20 [00:04<00:37,  2.06s/it]Epoch: 3 | train_loss: 1.0331 | train_acc: 0.5260 | test_loss: 1.0651 | test_acc: 0.2798
 15% 3/20 [00:06<00:38,  2.26s/it]Epoch: 4 | train_loss: 0.9631 | train_acc: 0.5858 | test_loss: 1.0616 | test_acc: 0.2422
 20% 4/20 [00:08<00:32,  2.06s/it]Epoch: 5 | train_loss: 0.9023 | train_acc: 0.6085 | test_loss: 1.1170 | test_acc: 0.2955
 25% 5/20 [00:10<00:29,  1.96s/it]Epoch: 6 | train_loss: 0.8589 | train_acc: 0.6219 | test_loss: 1.1661 | test_acc: 0.5135
 30% 6/20 [00:12<00:31,  2.23s/it]Epoch: 7 | tra

In [ ]:
import torch
saved_model = torch.load("models/05_going_modular_script_mode_tinyvgg_model.pth")
saved_model

OrderedDict([('conv_block_1.0.weight',
              tensor([[[[-3.4652e-02, -1.3061e-01, -4.8222e-02],
                        [ 1.5150e-01,  4.1626e-02,  1.0120e-02],
                        [-1.0081e-01, -8.9378e-02,  1.2344e-01]],
              
                       [[-1.0888e-01, -7.6788e-03,  7.6478e-02],
                        [ 1.1496e-01,  1.5195e-01, -1.1728e-01],
                        [ 3.4599e-02, -9.7465e-02, -6.1856e-02]],
              
                       [[ 1.5206e-01, -2.0577e-01,  1.2081e-01],
                        [-4.9606e-03, -5.6250e-02,  9.0134e-02],
                        [-1.3495e-01,  6.6681e-02, -1.6484e-01]]],
              
              
                      [[[ 5.3360e-02,  1.0139e-01, -6.1184e-02],
                        [-3.0128e-02,  1.7949e-02,  1.3790e-01],
                        [-1.3679e-01, -1.7212e-01, -2.0506e-01]],
              
                       [[-1.1709e-01,  1.3730e-01,  7.1237e-02],
                        [-6.7209e-02

In [ ]:
%%writefile going_modular/predict.py

"""
Takes a custom image path and use a model that was saved earlier to predict on it
"""

import os
import torch
import argparse
from argparse import ArgumentParser
from pathlib import Path
import torchvision
from torchvision import transforms
import model_builder
import data_setup # Import data_setup


#setting up default custom image_path
# Download custom image
import requests

# Setup custom image path
data_path = Path("data/")
custom_image_path_in_script = data_path / "04-pizza-dad.jpeg"

# Download the image if it doesn't already exist
if not custom_image_path_in_script.is_file():
    with open(custom_image_path_in_script, "wb") as f:
        # When downloading from GitHub, need to use the "raw" file link
        request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/04-pizza-dad.jpeg")
        print(f"Downloading {custom_image_path_in_script}...")
        f.write(request.content)
else:
    print(f"{custom_image_path_in_script} already exists, skipping download.")


#settinng up argparser

parser = ArgumentParser(description = "Argument for custom image path")
parser.add_argument("custom_image_path", default = custom_image_path_in_script, help="this should be the path to a custom image to predict on, eg")

args = parser.parse_args()

#settingg device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms (need to be the same as training)
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders to get class_names (batch_size doesn't matter for getting class_names)
# Assuming train_dir and test_dir are consistent with how the model was trained
# We will use the same paths as in train.py for consistency
_, _, class_names = data_setup.create_dataloaders(
    train_dir="data/pizza_steak_sushi/train",
    test_dir="data/pizza_steak_sushi/test",
    transform=data_transform,
    batch_size=32
)

#setting up model
saved_model = model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=10,
    output_shape=len(class_names) # Use dynamic output_shape based on class_names
)
saved_model.load_state_dict(torch.load("models/05_going_modular_script_mode_tinyvgg_model.pth"))

saved_model.to(device)

# Load in custom image and convert the tensor values to float32
custom_image = torchvision.io.read_image(str(args.custom_image_path)).type(torch.float32)

# Divide the image pixel values by 255 to get them between [0, 1]
custom_image = custom_image / 255.

# Create transform pipleine to resize image
custom_image_transform = transforms.Compose([
    transforms.Resize((64, 64)),
])

# Transform target image
custom_image_transformed = custom_image_transform(custom_image)

##Predicting on model
saved_model.eval()
with torch.inference_mode():
  y_custom_pred_logits = saved_model(custom_image_transformed.unsqueeze(dim=0))
  y_custom_pred = torch.argmax(torch.softmax(y_custom_pred_logits, dim=1), dim=1)

prediction = class_names[y_custom_pred.item()]

print(f"Prediction: {prediction}")

Writing going_modular/predict.py


In [ ]:
# Download custom image
import requests

# Setup custom image path
custom_image_path = data_path / "random_pizza.jpeg"

# Download the image if it doesn't already exist
if not custom_image_path.is_file():
    with open(custom_image_path, "wb") as f:
        # When downloading from GitHub, need to use the "raw" file link
        request = requests.get("https://media.istockphoto.com/id/1442417585/photo/person-getting-a-piece-of-cheesy-pepperoni-pizza.jpg?s=1024x1024&w=is&k=20&c=faq73viCFGvfpKxcBuHcOI8kyT99B-p-jScipke-VuQ=")
        print(f"Downloading {custom_image_path}...")
        f.write(request.content)
else:
    print(f"{custom_image_path} already exists, skipping download.")

In [ ]:
# Execute predict.py with the correct image path
!python going_modular/predict.py {str(custom_image_path)}

Prediction: sushi
